# First examples of using TF for modeling
Before we start with simplest example possible, we need to explain role of variables in modeling:

## Variables and gradient computation
Variables are typically used to compute gradients. However, can be used for any operation, when declared

In [2]:
import tensorflow as tf
import numpy as np
A = tf.Variable([[3, 2],[5, 2]])
B = tf.constant([[9, 5],[1, 3]])
AB_conc = tf.concat(values=[A, B], axis=1)
print('Adding B\'s columns to A:\n', AB_conc)

Adding B's columns to A:
 tf.Tensor(
[[3 2 9 5]
 [5 2 1 3]], shape=(2, 4), dtype=int32)


But as we said the main purpose of variables is to use them for computation of gradient. This is easy with predefined functions that compute gradiaent for any defined function/computational graph. One of these functions is Computing gradient 'gradient tape' which is symbolic gradient computational procedure. Notice that it computes gradiaent (derivative at a given point

In [ ]:
x = tf.Variable(3.0)
with tf.GradientTape() as tape:
  y = x**2
dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


Now let's do linear predictor with weights $W=\begin{pmatrix}-1.25&0.56\\-0.62&-0.39\\1.82&-0.25\end{pmatrix}$ and its gradient at a point $\vec{x}\begin{pmatrix}1\\2\\3\\\end{pmatrix}$:

In [ ]:
w = tf.Variable(name='w', initial_value=[[-1.25,  0.56 ],
       [-0.62 , -0.39],
       [ 1.82 , -0.25]])
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
    y = x @ w + b
    loss = tf.reduce_mean(y**2)
[dl_dw, dl_db] = tape.gradient(loss, [w, b])
print(w)
print(b)
print('x=',x)
print('y=',y)
print('l=',loss)
print('dw=', dl_dw)
print('db=', dl_db)
print('shapes:')
print(w.shape)
print(dl_dw.shape)

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[-1.25,  0.56],
       [-0.62, -0.39],
       [ 1.82, -0.25]], dtype=float32)>
<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>
x= [[1.0, 2.0, 3.0]]
y= tf.Tensor([[ 2.97       -0.96999997]], shape=(1, 2), dtype=float32)
l= tf.Tensor(4.8809, shape=(), dtype=float32)
dw= tf.Tensor(
[[ 2.97       -0.96999997]
 [ 5.94       -1.9399999 ]
 [ 8.91       -2.9099998 ]], shape=(3, 2), dtype=float32)
db= tf.Tensor([ 2.97       -0.96999997], shape=(2,), dtype=float32)
shapes:
(3, 2)
(3, 2)


# tf functions
tf.function can be used to make computational graphs out of your programs. It is a transformation tool that creates Python-independent dataflow graphs out of your Python code. This helps to create performant and portable models, and it is necessary to save them. It allows any type of data input so it is independent of what kind of data you use.

Definition starts with @tf.fuction and then is defined the same way any python function is defined.
Warning: you cannot put breakpoints inside tensorflow functions and even inside python functions that are encapsulated inside tensorflow functions 

In [ ]:
W = tf.Variable(tf.ones(shape=[2,2]), name="W")
b = tf.Variable(tf.ones(shape=(2)), name="b")
# Use the decorator
@tf. function
def forward(x):
    s=tf.linalg.matvec(W,x)+b
    return s
#t=tf.Variable([1.0,1.0],shape=(2), name='vector')
out_a = forward(tf.constant([1.0,1.0], shape=2))
print(out_a) 

tf.Tensor([3. 3.], shape=(2,), dtype=float32)


## Linear Neuron
We create class with two methods: 

    - forward computation
    - backward propagation 
 We need to import tf and numpy.
 
Initalization of an oject includes some values of inital weights and initial bias. But we do not know yet what the layer will be so we do not define the shape th

In [ ]:
import tensorflow as tf
import numpy as np
import math

class Neuron:
    
    def __init__(self):
        self._b = tf.Variable(0.,shape=tf.TensorShape(None))
        self._W = tf.Variable(1., shape=tf.TensorShape(None))
        self._r =tf.Variable(0.5,shape=tf.TensorShape(None))
        
#forward computation takes weights and bias as input and returns predicted value
        
    def forward(self, inp, weights, bias):
        self._b.assign(bias)
        self._W.assign(weights)
        y_hat = tf.reduce_sum(self._W @ inp, 1) + self._b #axis in reduce sum is 1
        print('forward predicted value is:', y_hat)
        return y_hat
#gradient requires initial values of weights and bias
# but after one run on input instance these values are not needed so  they are optional with default set to None
# we compute eucledean loss as we described in class
# we compute then gradient using gradient tap as in simple exampel

    def gradient(self, x, y, w = None, b = None):
        if w == None:
            w = self._W
        if b == None:
            b= self._b
        with tf.GradientTape() as g:
            loss = 0.5*(y-self.forward(x,w,b ))**2
        print("Loss: ", loss)
        [dL_dW,dL_db] = g.gradient(loss, [self._W,self._b])
        return dL_dW,dL_db
    
    def update(self, grad_w, grad_b, r = None):
        if r != None:
            self._r.assign(r)
        rate = self._r
        self._W.assign_sub(rate * grad_w)
        self._b.assign_sub(rate * grad_b)
        return self._W,self._b

Initalize neuron. Initial weights, bias and first (x,y) pair must be given to compute first gradient

In [ ]:
#initalize neuron
my_neuron = Neuron()
w_0 = [[1.0,2.0]]
b = 0.0
[x,y] = [[[1.0],[1.0]], 2.7]
g_w,g_b = my_neuron.gradient(x,y,w_0,b)
print('gradient in weights =', g_w,'\n','bias gradient=',g_b)


forward predicted value is: tf.Tensor([3.], shape=(1,), dtype=float32)
Loss:  tf.Tensor([0.04499999], shape=(1,), dtype=float32)
gradient in weights = tf.Tensor([[0.29999995 0.29999995]], shape=(1, 2), dtype=float32) 
 bias gradient= tf.Tensor(0.29999995, shape=(), dtype=float32)


Now we can compute new weights using learning rate that we must define for the first time. 

In [ ]:
w1,b1 = my_neuron.update(g_w,g_b,0.5)
print('new weights =', w1,'\n','new bias=',b1)

new weights = <tf.Variable 'Variable:0' shape=<unknown> dtype=float32, numpy=array([[0.85, 1.85]], dtype=float32)> 
 new bias= <tf.Variable 'Variable:0' shape=<unknown> dtype=float32, numpy=-0.14999998>


Now with the new data we no longer need w,b,r we have r that doesn't change and we have new weights that are stored after last update so we just do next gradient computation and next update

In [ ]:
[x1,y1]=[[[2.0],[1.0]], 3.5]
g_w1,g_b1 = my_neuron.gradient(x1,y1)
print('new gradient in weights =', g_w1,'\n','new bias gradient=',g_b1)
w2,b2 = my_neuron.update(g_w1,g_b1)
print('new weights =', w2,'\n','new bias=',b2)

forward predicted value is: tf.Tensor([3.4], shape=(1,), dtype=float32)
Loss:  tf.Tensor([0.00499999], shape=(1,), dtype=float32)
new gradient in weights = tf.Tensor([[-0.19999981 -0.0999999 ]], shape=(1, 2), dtype=float32) 
 new bias gradient= tf.Tensor(-0.099999905, shape=(), dtype=float32)
new weights = <tf.Variable 'Variable:0' shape=<unknown> dtype=float32, numpy=array([[0.9499999, 1.9      ]], dtype=float32)> 
 new bias= <tf.Variable 'Variable:0' shape=<unknown> dtype=float32, numpy=-0.100000024>


## First realistic example: linear regression
Before we do anything that involves data we need to:

    - import necessary modules
    - check if we are working in Colab or locally
    - if we are working in Colab mount google drive
    - if we are in colab and file exists in expected folderwe can proceed 
    - Else (we are not in colab or files do not exist on google drive) we must check if files exist in expected folder locally. If no data file we exit
    - If we work in colab we need to upload upload data files to expected folder

But even before that we need to import modules: pandas to read csv files, math for mathematics, and from numpy import default random number generator 
from numpy.random import default_rng

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
import os
from numpy.random import default_rng

Are we in colab or locally?

In [2]:
try:
    from google.colab import drive
    IN_COLAB=True
except:
    IN_COLAB=False

We now need to mount google drive and upload file

In [3]:
if IN_COLAB:
  drive.mount('/content/gdrive')
  base_dir ='/content/gdrive/My Drive/courses/Deep Learning/data'
  if not os.path.exists(base_dir):
    os.mkdir(base_dir)
  %cd /content/gdrive/My Drive/courses/Deep Learning/data
else:
  base_dir = os.getcwd()+'\data'
fName = input("Enter a filename (must be in standard working folder):")
file = os.path.join(base_dir, fName)
try:
  data = open(file, 'r')
  ferror = False 
except FileNotFoundError:
  ferror=True
if IN_COLAB and ferror:
    from google.colab import files
    files.upload()
else:
  if ferror:
    rep=True
    while rep: 
      fName1 = input("Enter complete path to the data or q (to quit):")
      if fName1 == 'q':
        exit
      else:
        try:
          data = open(fName1, 'r')
          rep=False
        except:
          print('incorrect file name')

Enter a filename (must be in standard working folder):housing.txt


Now we can finally read data in! We have 20433 rows in our input file

In [4]:
data = pd.read_csv(file, header=None, names=['longitude','latitude','housing_median_age','total_rooms',\
                                             'total_bedrooms','population','households','median_income',\
                                             'median_house_value','ocean_proximity'])
data.dropna(axis=0,how='any',thresh=None,subset=None,inplace=True)
cols = data.shape[1]
rows = data.shape[0]
print(rows)


20433


Next we define the class maltivariate regression.

1. Initialization: it is just like in our simple neuron, except we allow to initialize bias be either random (by defualt) or one or all 0. Weights are initalized to single weight = 1 with actual value/shape being unknow until first called. Clearly both wights and bias are variables. What is new is that we define size here. Size is the size of one batch of data. The latter is a protion of data that will be processed  together as 1 unit of data.
2. Predict: just like forward before returns  $W\times \vec{x}+b$ where $W,b$ are being the procedure inputs just like inout $x$. Note that $x$ this time can be anything (e.g. a matrix - bacth of input vectors) not nesessarily 1 vector the only requirment is that it matched dimesion of $W$
3. Since $x$ could be a batch we compute instead of square of differences of 1 value from its target mean square of differences of a vector of targets from a vector of predicted values
4. Update: No changes in gradient computation but this time we include update itself as par tof the procedure($W_{new}=W_{old}-rate*gradient_W$ and $b_{new}=b_{old}-rate*gradient_b$)
5. train procedure is new. It take as input the total array of all data X and the total array of target variables y, and learning rate and number of epochs - number of cycles used to train the regression on ALL data. The procedure splits data into batches and train the weights on every batch starting with the first batch. This consitutes one cycle. It repeats traing on as many cycles as the argument 'epochs'  


In [5]:

class MultivariateLinearRegression:
    def __init__(self, initializer='random'):
        if initializer == 'ones':
            self.var = 1.
        elif initializer == 'zeros':
            self.var = 0.
        elif initializer == 'random':
            self.var = tf.random.uniform(shape=[], minval=0., maxval=1.)

        self.W = tf.Variable(1., shape=tf.TensorShape(None))
        self.b = tf.Variable(self.var)
        self.size = 500


    def predict(self, x):
        return tf.reduce_sum(self.W * x, 1) + self.b

    def mse(self, true, predicted):
        return tf.reduce_mean(tf.square(true - predicted))

    def update(self, X, y, learning_rate):
        with tf.GradientTape() as g:
            loss = self.mse(y, tf.reduce_sum(self.W * X, 1) + self.b)

        print("Loss: ", loss)

        W_grad,b_grad = g.gradient(loss, [self.W,self.b])

        self.W.assign_sub(learning_rate * W_grad)
        self.b.assign_sub(learning_rate * b_grad)

    def train(self, X, y, learning_rate=0.01, epochs=5):

        if len(X.shape) == 1:
            X = tf.reshape(X, [X.shape[0], 1])

        if X.shape[0] > self.size:
            batch_start_idx = 0
            batch_max=math.floor(X.shape[0]/self.size)
        else:
            batch_start_idx = -1

        self.W.assign([self.var] * X.shape[-1])

        for i in range(epochs):
            print("Epoch: ", i)
            if batch_start_idx < 0:
                Inp = X
                y_inp=y
            else:
                batch_start_idx=i % batch_max
                batch_end_idx=batch_start_idx+self.size
                Inp =X[batch_start_idx:batch_end_idx,:]
                y_inp=y[batch_start_idx:batch_end_idx]
            self.update(Inp, y_inp, learning_rate)

Random number grenerator is used to separate 3/4 data for training from 1/4 for testing (lines 4 to 10). Then we normalize the data in every column, .e.compute mean and std for  every column of training data and compute $\frac{x-\overline{x}}{\sigma{x}}$ where $\overline{x}$ is the mean of column $x$ and $\sigma{x}$ is standard deviation of this column. This is done in lines 12-17. Then we initialize the model (instantiate the class Multivariate regression) in line 19 and then we train the model in line 21. Then we sttandardize test data in lines 23-25. Predict the targets for test data in line 27. Reverse standartization of predicted value by  

In [18]:
X = np.array(data.iloc[:,0:cols-2],np.float32)
y = np.array(data.iloc[:,cols-2:cols-1],np.float32)
rng = default_rng()
#select training and testing data
train_idx = rng.choice(rows, size = math.ceil(rows*3/4), replace=False)
x_train=X[train_idx,:]
y_train=y[train_idx,:]
test_idx=np.setdiff1d(np.array(range(0,rows)),train_idx)
x_test=X[test_idx,:]
y_test=y[test_idx,:]
#Standardize train data 
mean_target = y_train.mean(axis=0)
std_target = y_train.std(axis=0)
mean_feat = x_train.mean(axis=0)
std_feat = x_train.std(axis=0)
x_train_norm = (x_train-mean_feat)/std_feat
y_train_norm = (y_train-mean_target)/std_target
#initalize model
linear_model = MultivariateLinearRegression('random') 
#train model
linear_model.train(x_train_norm, y_train_norm, learning_rate=0.1, epochs=4000)
# standardize test data
mean_feat_test = x_test.mean(axis=0)
std_feat_test = x_test.std(axis=0)
x_test_norm = (x_test-mean_feat_test)/std_feat_test
#Predict target values
pred = linear_model.predict(x_test_norm)
# reverse standardization
std_target_test = y_test.std(axis=0)
mean_target_test = y_test.mean(axis=0)
pred *= std_target_test
pred += mean_target_test
#print(pred)
acc=1-(tf.reduce_mean(abs(y_test-pred))/tf.reduce_mean(y_test))
print('accuracy is: ',acc.numpy())

Epoch:  0
Loss:  tf.Tensor(6.862136, shape=(), dtype=float32)
Epoch:  1
Loss:  tf.Tensor(1.6830711, shape=(), dtype=float32)
Epoch:  2
Loss:  tf.Tensor(1.4518118, shape=(), dtype=float32)
Epoch:  3
Loss:  tf.Tensor(1.3075088, shape=(), dtype=float32)
Epoch:  4
Loss:  tf.Tensor(1.2115574, shape=(), dtype=float32)
Epoch:  5
Loss:  tf.Tensor(1.1437595, shape=(), dtype=float32)
Epoch:  6
Loss:  tf.Tensor(1.097346, shape=(), dtype=float32)
Epoch:  7
Loss:  tf.Tensor(1.0659066, shape=(), dtype=float32)
Epoch:  8
Loss:  tf.Tensor(1.042816, shape=(), dtype=float32)
Epoch:  9
Loss:  tf.Tensor(1.0264925, shape=(), dtype=float32)
Epoch:  10
Loss:  tf.Tensor(1.0191661, shape=(), dtype=float32)
Epoch:  11
Loss:  tf.Tensor(1.0093856, shape=(), dtype=float32)
Epoch:  12
Loss:  tf.Tensor(1.0015708, shape=(), dtype=float32)
Epoch:  13
Loss:  tf.Tensor(0.9840346, shape=(), dtype=float32)
Epoch:  14
Loss:  tf.Tensor(0.9802696, shape=(), dtype=float32)
Epoch:  15
Loss:  tf.Tensor(0.975522, shape=(), dtype

Loss:  tf.Tensor(0.9371898, shape=(), dtype=float32)
Epoch:  174
Loss:  tf.Tensor(0.93688935, shape=(), dtype=float32)
Epoch:  175
Loss:  tf.Tensor(0.93497723, shape=(), dtype=float32)
Epoch:  176
Loss:  tf.Tensor(0.9274122, shape=(), dtype=float32)
Epoch:  177
Loss:  tf.Tensor(0.93021333, shape=(), dtype=float32)
Epoch:  178
Loss:  tf.Tensor(0.93003637, shape=(), dtype=float32)
Epoch:  179
Loss:  tf.Tensor(0.9304498, shape=(), dtype=float32)
Epoch:  180
Loss:  tf.Tensor(0.953312, shape=(), dtype=float32)
Epoch:  181
Loss:  tf.Tensor(0.9515104, shape=(), dtype=float32)
Epoch:  182
Loss:  tf.Tensor(0.9513537, shape=(), dtype=float32)
Epoch:  183
Loss:  tf.Tensor(0.9515487, shape=(), dtype=float32)
Epoch:  184
Loss:  tf.Tensor(0.9523496, shape=(), dtype=float32)
Epoch:  185
Loss:  tf.Tensor(0.95228076, shape=(), dtype=float32)
Epoch:  186
Loss:  tf.Tensor(0.95182383, shape=(), dtype=float32)
Epoch:  187
Loss:  tf.Tensor(0.9519486, shape=(), dtype=float32)
Epoch:  188
Loss:  tf.Tensor(0.9

Epoch:  325
Loss:  tf.Tensor(0.933861, shape=(), dtype=float32)
Epoch:  326
Loss:  tf.Tensor(0.92631614, shape=(), dtype=float32)
Epoch:  327
Loss:  tf.Tensor(0.9291398, shape=(), dtype=float32)
Epoch:  328
Loss:  tf.Tensor(0.9289829, shape=(), dtype=float32)
Epoch:  329
Loss:  tf.Tensor(0.9294157, shape=(), dtype=float32)
Epoch:  330
Loss:  tf.Tensor(0.95217896, shape=(), dtype=float32)
Epoch:  331
Loss:  tf.Tensor(0.95040935, shape=(), dtype=float32)
Epoch:  332
Loss:  tf.Tensor(0.9502799, shape=(), dtype=float32)
Epoch:  333
Loss:  tf.Tensor(0.95049894, shape=(), dtype=float32)
Epoch:  334
Loss:  tf.Tensor(0.95132476, shape=(), dtype=float32)
Epoch:  335
Loss:  tf.Tensor(0.95127755, shape=(), dtype=float32)
Epoch:  336
Loss:  tf.Tensor(0.95084244, shape=(), dtype=float32)
Epoch:  337
Loss:  tf.Tensor(0.9509869, shape=(), dtype=float32)
Epoch:  338
Loss:  tf.Tensor(0.9505699, shape=(), dtype=float32)
Epoch:  339
Loss:  tf.Tensor(0.95060015, shape=(), dtype=float32)
Epoch:  340
Loss: 

Loss:  tf.Tensor(0.92837, shape=(), dtype=float32)
Epoch:  467
Loss:  tf.Tensor(0.9288116, shape=(), dtype=float32)
Epoch:  468
Loss:  tf.Tensor(0.9291189, shape=(), dtype=float32)
Epoch:  469
Loss:  tf.Tensor(0.9285801, shape=(), dtype=float32)
Epoch:  470
Loss:  tf.Tensor(0.9415191, shape=(), dtype=float32)
Epoch:  471
Loss:  tf.Tensor(0.94443005, shape=(), dtype=float32)
Epoch:  472
Loss:  tf.Tensor(0.93688846, shape=(), dtype=float32)
Epoch:  473
Loss:  tf.Tensor(0.9359314, shape=(), dtype=float32)
Epoch:  474
Loss:  tf.Tensor(0.93567175, shape=(), dtype=float32)
Epoch:  475
Loss:  tf.Tensor(0.9337815, shape=(), dtype=float32)
Epoch:  476
Loss:  tf.Tensor(0.9262373, shape=(), dtype=float32)
Epoch:  477
Loss:  tf.Tensor(0.92906344, shape=(), dtype=float32)
Epoch:  478
Loss:  tf.Tensor(0.92890805, shape=(), dtype=float32)
Epoch:  479
Loss:  tf.Tensor(0.9293422, shape=(), dtype=float32)
Epoch:  480
Loss:  tf.Tensor(0.952092, shape=(), dtype=float32)
Epoch:  481
Loss:  tf.Tensor(0.9503

Epoch:  602
Loss:  tf.Tensor(0.9501935, shape=(), dtype=float32)
Epoch:  603
Loss:  tf.Tensor(0.9504152, shape=(), dtype=float32)
Epoch:  604
Loss:  tf.Tensor(0.9512439, shape=(), dtype=float32)
Epoch:  605
Loss:  tf.Tensor(0.95119894, shape=(), dtype=float32)
Epoch:  606
Loss:  tf.Tensor(0.9507655, shape=(), dtype=float32)
Epoch:  607
Loss:  tf.Tensor(0.950912, shape=(), dtype=float32)
Epoch:  608
Loss:  tf.Tensor(0.95049626, shape=(), dtype=float32)
Epoch:  609
Loss:  tf.Tensor(0.9505302, shape=(), dtype=float32)
Epoch:  610
Loss:  tf.Tensor(0.9538666, shape=(), dtype=float32)
Epoch:  611
Loss:  tf.Tensor(0.9524524, shape=(), dtype=float32)
Epoch:  612
Loss:  tf.Tensor(0.9509111, shape=(), dtype=float32)
Epoch:  613
Loss:  tf.Tensor(0.93805104, shape=(), dtype=float32)
Epoch:  614
Loss:  tf.Tensor(0.937912, shape=(), dtype=float32)
Epoch:  615
Loss:  tf.Tensor(0.93575543, shape=(), dtype=float32)
Epoch:  616
Loss:  tf.Tensor(0.9283617, shape=(), dtype=float32)
Epoch:  617
Loss:  tf.T

...

Epoch:  3343
Loss:  tf.Tensor(0.9380495, shape=(), dtype=float32)
Epoch:  3344
Loss:  tf.Tensor(0.9379103, shape=(), dtype=float32)
Epoch:  3345
Loss:  tf.Tensor(0.93575376, shape=(), dtype=float32)
Epoch:  3346
Loss:  tf.Tensor(0.92836, shape=(), dtype=float32)
Epoch:  3347
Loss:  tf.Tensor(0.928802, shape=(), dtype=float32)
Epoch:  3348
Loss:  tf.Tensor(0.92910904, shape=(), dtype=float32)
Epoch:  3349
Loss:  tf.Tensor(0.92857057, shape=(), dtype=float32)
Epoch:  3350
Loss:  tf.Tensor(0.9415101, shape=(), dtype=float32)
Epoch:  3351
Loss:  tf.Tensor(0.9444213, shape=(), dtype=float32)
Epoch:  3352
Loss:  tf.Tensor(0.93687963, shape=(), dtype=float32)
Epoch:  3353
Loss:  tf.Tensor(0.9359224, shape=(), dtype=float32)
Epoch:  3354
Loss:  tf.Tensor(0.9356628, shape=(), dtype=float32)
Epoch:  3355
Loss:  tf.Tensor(0.93377256, shape=(), dtype=float32)
Epoch:  3356
Loss:  tf.Tensor(0.9262284, shape=(), dtype=float32)
Epoch:  3357
Loss:  tf.Tensor(0.92905486, shape=(), dtype=float32)
Epoch: 

Loss:  tf.Tensor(0.951197, shape=(), dtype=float32)
Epoch:  3486
Loss:  tf.Tensor(0.9507636, shape=(), dtype=float32)
Epoch:  3487
Loss:  tf.Tensor(0.95091015, shape=(), dtype=float32)
Epoch:  3488
Loss:  tf.Tensor(0.9504945, shape=(), dtype=float32)
Epoch:  3489
Loss:  tf.Tensor(0.9505285, shape=(), dtype=float32)
Epoch:  3490
Loss:  tf.Tensor(0.953865, shape=(), dtype=float32)
Epoch:  3491
Loss:  tf.Tensor(0.9524509, shape=(), dtype=float32)
Epoch:  3492
Loss:  tf.Tensor(0.95090944, shape=(), dtype=float32)
Epoch:  3493
Loss:  tf.Tensor(0.9380495, shape=(), dtype=float32)
Epoch:  3494
Loss:  tf.Tensor(0.9379102, shape=(), dtype=float32)
Epoch:  3495
Loss:  tf.Tensor(0.93575376, shape=(), dtype=float32)
Epoch:  3496
Loss:  tf.Tensor(0.9283599, shape=(), dtype=float32)
Epoch:  3497
Loss:  tf.Tensor(0.928802, shape=(), dtype=float32)
Epoch:  3498
Loss:  tf.Tensor(0.92910904, shape=(), dtype=float32)
Epoch:  3499
Loss:  tf.Tensor(0.92857057, shape=(), dtype=float32)
Epoch:  3500
Loss:  t

Epoch:  3629
Loss:  tf.Tensor(0.9293341, shape=(), dtype=float32)
Epoch:  3630
Loss:  tf.Tensor(0.952081, shape=(), dtype=float32)
Epoch:  3631
Loss:  tf.Tensor(0.9503165, shape=(), dtype=float32)
Epoch:  3632
Loss:  tf.Tensor(0.95019126, shape=(), dtype=float32)
Epoch:  3633
Loss:  tf.Tensor(0.95041317, shape=(), dtype=float32)
Epoch:  3634
Loss:  tf.Tensor(0.95124185, shape=(), dtype=float32)
Epoch:  3635
Loss:  tf.Tensor(0.951197, shape=(), dtype=float32)
Epoch:  3636
Loss:  tf.Tensor(0.9507636, shape=(), dtype=float32)
Epoch:  3637
Loss:  tf.Tensor(0.95091015, shape=(), dtype=float32)
Epoch:  3638
Loss:  tf.Tensor(0.9504945, shape=(), dtype=float32)
Epoch:  3639
Loss:  tf.Tensor(0.9505285, shape=(), dtype=float32)
Epoch:  3640
Loss:  tf.Tensor(0.953865, shape=(), dtype=float32)
Epoch:  3641
Loss:  tf.Tensor(0.95245075, shape=(), dtype=float32)
Epoch:  3642
Loss:  tf.Tensor(0.95090955, shape=(), dtype=float32)
Epoch:  3643
Loss:  tf.Tensor(0.9380495, shape=(), dtype=float32)
Epoch: 

Epoch:  3773
Loss:  tf.Tensor(0.9359224, shape=(), dtype=float32)
Epoch:  3774
Loss:  tf.Tensor(0.9356628, shape=(), dtype=float32)
Epoch:  3775
Loss:  tf.Tensor(0.93377256, shape=(), dtype=float32)
Epoch:  3776
Loss:  tf.Tensor(0.9262284, shape=(), dtype=float32)
Epoch:  3777
Loss:  tf.Tensor(0.92905486, shape=(), dtype=float32)
Epoch:  3778
Loss:  tf.Tensor(0.9288998, shape=(), dtype=float32)
Epoch:  3779
Loss:  tf.Tensor(0.929334, shape=(), dtype=float32)
Epoch:  3780
Loss:  tf.Tensor(0.952081, shape=(), dtype=float32)
Epoch:  3781
Loss:  tf.Tensor(0.9503164, shape=(), dtype=float32)
Epoch:  3782
Loss:  tf.Tensor(0.95019126, shape=(), dtype=float32)
Epoch:  3783
Loss:  tf.Tensor(0.95041317, shape=(), dtype=float32)
Epoch:  3784
Loss:  tf.Tensor(0.95124185, shape=(), dtype=float32)
Epoch:  3785
Loss:  tf.Tensor(0.951197, shape=(), dtype=float32)
Epoch:  3786
Loss:  tf.Tensor(0.9507636, shape=(), dtype=float32)
Epoch:  3787
Loss:  tf.Tensor(0.95091015, shape=(), dtype=float32)
Epoch: 

Loss:  tf.Tensor(0.92910904, shape=(), dtype=float32)
Epoch:  3919
Loss:  tf.Tensor(0.9285706, shape=(), dtype=float32)
Epoch:  3920
Loss:  tf.Tensor(0.9415101, shape=(), dtype=float32)
Epoch:  3921
Loss:  tf.Tensor(0.9444213, shape=(), dtype=float32)
Epoch:  3922
Loss:  tf.Tensor(0.93687963, shape=(), dtype=float32)
Epoch:  3923
Loss:  tf.Tensor(0.9359224, shape=(), dtype=float32)
Epoch:  3924
Loss:  tf.Tensor(0.9356628, shape=(), dtype=float32)
Epoch:  3925
Loss:  tf.Tensor(0.9337725, shape=(), dtype=float32)
Epoch:  3926
Loss:  tf.Tensor(0.9262284, shape=(), dtype=float32)
Epoch:  3927
Loss:  tf.Tensor(0.929055, shape=(), dtype=float32)
Epoch:  3928
Loss:  tf.Tensor(0.9288998, shape=(), dtype=float32)
Epoch:  3929
Loss:  tf.Tensor(0.929334, shape=(), dtype=float32)
Epoch:  3930
Loss:  tf.Tensor(0.952081, shape=(), dtype=float32)
Epoch:  3931
Loss:  tf.Tensor(0.9503165, shape=(), dtype=float32)
Epoch:  3932
Loss:  tf.Tensor(0.9501913, shape=(), dtype=float32)
Epoch:  3933
Loss:  tf.T